In [3]:
# !mkdir -p ~/.kaggle
# !cp kaggle.json ~/.kaggle/
# !chmod 600 ~/.kaggle/kaggle.json

# !kaggle competitions download -c deepfake-detection-challenge

# !unzip deepfake-detection-challenge.zip

deepfake-detection-challenge.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  deepfake-detection-challenge.zip
replace sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace test_videos/aassnaulhq.mp4? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace test_videos/aayfryxljh.mp4? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [3]:
import os
import json
import cv2
import numpy as np
from scipy.signal import butter, filtfilt
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from joblib import Parallel, delayed
import multiprocessing

def extract_frames_and_detect_faces(video_path, frame_skip=5, scale_factor=0.5):
    # Load the pre-trained face detector inside the function
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    cap = cv2.VideoCapture(video_path)
    frames = []
    frame_count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        if frame_count % frame_skip == 0:
            # Resize frame for faster processing
            frame = cv2.resize(frame, (0, 0), fx=scale_factor, fy=scale_factor)
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            faces = face_cascade.detectMultiScale(gray, 1.1, 4)
            for (x, y, w, h) in faces:
                roi_color = frame[y:y+h, x:x+w]
                frames.append(roi_color)
        frame_count += 1
    cap.release()
    return frames

def extract_green_channel_signal(frames):
    signals = []
    for frame in frames:
        height, width, _ = frame.shape
        forehead_region = frame[:height//3, :]
        green_channel = forehead_region[:, :, 1]
        mean_green_value = np.mean(green_channel)
        signals.append(mean_green_value)
    return signals

def bandpass_filter(signal, lowcut, highcut, fs, order=5):
    if len(signal) <= 33:
        return signal
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')
    y = filtfilt(b, a, signal)
    return y

def create_spatiotemporal_map(signal, width, height):
    spatiotemporal_map = np.zeros((height, width))
    signal_length = len(signal)
    for i in range(height):
        for j in range(width):
            if i < signal_length:
                spatiotemporal_map[i, j] = signal[i]
    return spatiotemporal_map

def process_video(video_path, map_width=50, lowcut=0.8, highcut=2.5, fs=30, fixed_height=300, frame_skip=5, scale_factor=0.5):
    frames = extract_frames_and_detect_faces(video_path, frame_skip, scale_factor)
    if len(frames) == 0:
        return None
    green_channel_signal = extract_green_channel_signal(frames)
    filtered_signal = bandpass_filter(green_channel_signal, lowcut, highcut, fs)
    map_height = len(filtered_signal)
    spatiotemporal_map = create_spatiotemporal_map(filtered_signal, map_width, map_height)
    spatiotemporal_map = cv2.resize(spatiotemporal_map, (map_width, fixed_height))
    return spatiotemporal_map

def process_videos_in_folder_parallel(folder_path, metadata_path, n_jobs=-1):
    with open(metadata_path, 'r') as f:
        metadata = json.load(f)
    
    video_files = [filename for filename in os.listdir(folder_path) if filename.endswith('.mp4') and filename in metadata]
    
    results = Parallel(n_jobs=n_jobs)(delayed(process_video)(
        os.path.join(folder_path, filename)) for filename in video_files)
    
    spatiotemporal_maps = [result for result in results if result is not None]
    labels = [1 if metadata[filename]['label'] == 'REAL' else 0 for filename in video_files if process_video(os.path.join(folder_path, filename)) is not None]
    
    return np.array(spatiotemporal_maps), np.array(labels)

# # Example folder path and metadata path
# folder_path = 'train_sample_videos'
# metadata_path = os.path.join(folder_path, 'metadata.json')
# spatiotemporal_maps, labels = process_videos_in_folder_parallel(folder_path, metadata_path)

# # Add channel dimension for grayscale images
# spatiotemporal_maps = np.expand_dims(spatiotemporal_maps, axis=-1)

# # Save spatiotemporal maps and labels to disk
# np.save('spatiotemporal_maps.npy', spatiotemporal_maps)
# np.save('labels.npy', labels)


In [4]:
# Load spatiotemporal maps and labels from disk
spatiotemporal_maps = np.load('spatiotemporal_maps.npy')
labels = np.load('labels.npy')

# Shuffle the data
indices = np.arange(len(labels))
np.random.shuffle(indices)
spatiotemporal_maps = spatiotemporal_maps[indices]
labels = labels[indices]

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(spatiotemporal_maps, labels, test_size=0.2, random_state=42)

def build_cnn_model(input_shape):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

input_shape = (300, 50, 1)  # Adjust the input shape if necessary
model = build_cnn_model(input_shape)

# Train the model
model.fit(X_train, y_train, epochs=25, batch_size=8, validation_data=(X_val, y_val))



c:\Users\kuruvadi.intern\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/25
 88/144 ━━━━━━━━━━━━━━━━━━━━ 6s 113ms/step - accuracy: 0.7536 - loss: 4.3465

KeyboardInterrupt: 

In [6]:
model_file_path = 'model.h5'
model.save(model_file_path)
model.save('my_model.keras')

In [21]:
def predict_video(video_path):
    spatiotemporal_map = process_video(video_path, fixed_height=300)
    if spatiotemporal_map is None:
        return 'Unable to process video'
    spatiotemporal_map = np.expand_dims(spatiotemporal_map, axis=0)
    spatiotemporal_map = np.expand_dims(spatiotemporal_map, axis=-1)
    prediction = model.predict(spatiotemporal_map)
    confidence_score = prediction[0][0]
    label = 'Real' if prediction[0][0] > 0.5 else 'Fake'
    return label,confidence_score

# Example prediction
test_video_path = r'dfdc_train_part_07\dfdc_train_part_7\vatsa.mp4'
print(predict_video(test_video_path))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
('Fake', 0.036299847)


In [29]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define a data generator with augmentation
datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

datagen.fit(X_train)

def build_cnn_model(input_shape):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

input_shape = (300, 50, 1)  # Adjust the input shape if necessary
model = build_cnn_model(input_shape)
# Train the model with augmented data
model.fit(datagen.flow(X_train, y_train, batch_size=8), epochs=20, validation_data=(X_val, y_val))


Epoch 1/20
39/39 ━━━━━━━━━━━━━━━━━━━━ 22s 320ms/step - accuracy: 0.7197 - loss: 2.8349 - val_accuracy: 0.8312 - val_loss: 0.9427
Epoch 2/20
39/39 ━━━━━━━━━━━━━━━━━━━━ 15s 336ms/step - accuracy: 0.7954 - loss: 0.6927 - val_accuracy: 0.8312 - val_loss: 0.5939
Epoch 3/20
39/39 ━━━━━━━━━━━━━━━━━━━━ 14s 332ms/step - accuracy: 0.8060 - loss: 0.6129 - val_accuracy: 0.8052 - val_loss: 0.5639
Epoch 4/20
39/39 ━━━━━━━━━━━━━━━━━━━━ 16s 374ms/step - accuracy: 0.7961 - loss: 0.6515 - val_accuracy: 0.8312 - val_loss: 0.4944
Epoch 5/20
39/39 ━━━━━━━━━━━━━━━━━━━━ 15s 350ms/step - accuracy: 0.8242 - loss: 0.4999 - val_accuracy: 0.8312 - val_loss: 0.5386
Epoch 6/20
39/39 ━━━━━━━━━━━━━━━━━━━━ 17s 393ms/step - accuracy: 0.8169 - loss: 0.5081 - val_accuracy: 0.8312 - val_loss: 0.5518
Epoch 7/20
39/39 ━━━━━━━━━━━━━━━━━━━━ 18s 389ms/step - accuracy: 0.8190 - loss: 0.5383 - val_accuracy: 0.8312 - val_loss: 0.5493
Epoch 8/20
39/39 ━━━━━━━━━━━━━━━━━━━━ 16s 350ms/step - accuracy: 0.7963 - loss: 0.5394 - val_accu

In [30]:
def predict_video(video_path):
    spatiotemporal_map = process_video(video_path, fixed_height=300)
    if spatiotemporal_map is None:
        return 'Unable to process video'
    spatiotemporal_map = np.expand_dims(spatiotemporal_map, axis=0)
    spatiotemporal_map = np.expand_dims(spatiotemporal_map, axis=-1)
    prediction = model.predict(spatiotemporal_map)
    confidence_score = prediction[0][0]
    label = 'Real' if prediction[0][0] > 0.5 else 'Fake'
    return label,confidence_score

# Example prediction
test_video_path = r'dfdc_train_part_07\dfdc_train_part_7\vatsa.mp4'
print(predict_video(test_video_path))

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
('Fake', 0.19952387)


In [28]:
model.save('my_models/model_data_augmentation.keras')

In [39]:
from tensorflow.keras.layers import Dropout

def build_regularized_cnn_model(input_shape):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))  
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

input_shape = (300, 50, 1) 
model = build_regularized_cnn_model(input_shape)
model.fit(datagen.flow(X_train, y_train, batch_size=16), epochs=20, validation_data=(X_val, y_val))

Epoch 1/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 26s 565ms/step - accuracy: 0.6542 - loss: 7.8683 - val_accuracy: 0.7662 - val_loss: 0.6241
Epoch 2/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 13s 497ms/step - accuracy: 0.7765 - loss: 0.6985 - val_accuracy: 0.8312 - val_loss: 0.6082
Epoch 3/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 12s 469ms/step - accuracy: 0.7881 - loss: 0.6493 - val_accuracy: 0.8312 - val_loss: 0.5912
Epoch 4/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 11s 440ms/step - accuracy: 0.7967 - loss: 0.6492 - val_accuracy: 0.8312 - val_loss: 0.5647
Epoch 5/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 11s 436ms/step - accuracy: 0.8131 - loss: 0.5572 - val_accuracy: 0.8312 - val_loss: 0.5464
Epoch 6/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 13s 494ms/step - accuracy: 0.7968 - loss: 0.5488 - val_accuracy: 0.8312 - val_loss: 0.5319
Epoch 7/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 12s 464ms/step - accuracy: 0.7967 - loss: 0.5511 - val_accuracy: 0.8312 - val_loss: 0.5229
Epoch 8/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 12s 513ms/step - accuracy: 0.8295 - loss: 0.5120 - val_accu

In [43]:
def predict_video(video_path):
    spatiotemporal_map = process_video(video_path, fixed_height=300)
    if spatiotemporal_map is None:
        return 'Unable to process video'
    spatiotemporal_map = np.expand_dims(spatiotemporal_map, axis=0)
    spatiotemporal_map = np.expand_dims(spatiotemporal_map, axis=-1)
    prediction = model.predict(spatiotemporal_map)
    confidence_score = prediction[0][0]
    label = 'Real' if prediction[0][0] > 0.5 else 'Fake'
    return label,confidence_score

# Example prediction
test_video_path = r'train_sample_videos\aelfnikyqj.mp4'
print(predict_video(test_video_path))

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
('Fake', 0.22052085)


In [45]:
# Load spatiotemporal maps and labels from disk
spatiotemporal_maps = np.load('spatiotemporal_maps.npy')
labels = np.load('labels.npy')

# Shuffle the data
indices = np.arange(len(labels))
np.random.shuffle(indices)
spatiotemporal_maps = spatiotemporal_maps[indices]
labels = labels[indices]

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(spatiotemporal_maps, labels, test_size=0.2, random_state=42)

datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

datagen.fit(X_train)

def build_cnn_model(input_shape):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

input_shape = (300, 50, 1)  # Adjust the input shape if necessary
model = build_cnn_model(input_shape)

model.fit(datagen.flow(X_train, y_train, batch_size=32), epochs=20, validation_data=(X_val, y_val))

Epoch 1/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 8s 184ms/step - accuracy: 0.7754 - loss: 15.6844 - val_accuracy: 0.8576 - val_loss: 0.7395
Epoch 2/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 6s 168ms/step - accuracy: 0.8381 - loss: 0.5873 - val_accuracy: 0.8576 - val_loss: 0.4521
Epoch 3/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 7s 176ms/step - accuracy: 0.8543 - loss: 0.4982 - val_accuracy: 0.8576 - val_loss: 0.4377
Epoch 4/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 6s 169ms/step - accuracy: 0.8636 - loss: 0.4703 - val_accuracy: 0.8576 - val_loss: 0.4284
Epoch 5/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 6s 167ms/step - accuracy: 0.8559 - loss: 0.4369 - val_accuracy: 0.8576 - val_loss: 0.4553
Epoch 6/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 6s 171ms/step - accuracy: 0.8647 - loss: 0.4485 - val_accuracy: 0.8576 - val_loss: 0.4331
Epoch 7/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 6s 168ms/step - accuracy: 0.8469 - loss: 0.4515 - val_accuracy: 0.8576 - val_loss: 0.4463
Epoch 8/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 6s 171ms/step - accuracy: 0.8585 - loss: 0.4497 - val_accuracy: 0

In [54]:
def predict_video(video_path):
    spatiotemporal_map = process_video(video_path, fixed_height=300)
    if spatiotemporal_map is None:
        return 'Unable to process video'
    spatiotemporal_map = np.expand_dims(spatiotemporal_map, axis=0)
    spatiotemporal_map = np.expand_dims(spatiotemporal_map, axis=-1)
    prediction = model.predict(spatiotemporal_map)
    confidence_score = prediction[0][0]
    label = 'Real' if prediction[0][0] > 0.5 else 'Fake'
    return label,confidence_score

# Example prediction
test_video_path = r'dfdc_train_part_07\dfdc_train_part_7\sudarshan.mp4'
print(predict_video(test_video_path))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
('Fake', 0.41947418)


In [52]:
model.save('my_models/model_1500.keras')

In [2]:
def extract_frames_and_detect_faces(video_path, frame_skip=5, scale_factor=0.5):
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    cap = cv2.VideoCapture(video_path)
    frames = []
    frame_count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        if frame_count % frame_skip == 0:
            frame = cv2.resize(frame, (0, 0), fx=scale_factor, fy=scale_factor)
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            faces = face_cascade.detectMultiScale(gray, 1.1, 4)
            for (x, y, w, h) in faces:
                roi_color = frame[y:y+h, x:x+w]
                frames.append(roi_color)
        frame_count += 1
    cap.release()
    return frames

# Adjust bandpass filter parameters to better capture physiological signals
def bandpass_filter(signal, lowcut, highcut, fs, order=5):
    if len(signal) <= 33:
        return signal
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')
    y = filtfilt(b, a, signal)
    return y

In [61]:
def process_video(video_path, frame_skip=5, fixed_height=300, fixed_width=50):
    frames = extract_frames_and_detect_faces(video_path, frame_skip)
    green_channel_signal = extract_green_channel_signal(frames)
    filtered_signal = bandpass_filter(green_channel_signal, lowcut=0.8, highcut=2.5, fs=30)
    if len(filtered_signal) < fixed_height:
        return None  # Skip video if not enough frames after filtering
    spatiotemporal_map = create_spatiotemporal_map(filtered_signal[:fixed_height], fixed_width, fixed_height)
    spatiotemporal_map = np.expand_dims(spatiotemporal_map, axis=-1)  # Add channel dimension
    return spatiotemporal_map



def process_videos_in_folder_parallel(folder_path, metadata_path, n_jobs=-1):
    with open(metadata_path, 'r') as f:
        metadata = json.load(f)
    
    video_files = [filename for filename in os.listdir(folder_path) if filename.endswith('.mp4') and filename in metadata]
    
    # Process videos in parallel
    results = Parallel(n_jobs=n_jobs)(delayed(process_video)(os.path.join(folder_path, filename)) for filename in video_files)
    
    spatiotemporal_maps = []
    labels = []
    
    for filename, result in zip(video_files, results):
        if result is not None:
            spatiotemporal_maps.append(result)
            label = 1 if metadata[filename]['label'] == 'REAL' else 0
            labels.append(label)
    
    return np.array(spatiotemporal_maps), np.array(labels)


# Adjusting shapes for training
spatiotemporal_maps_new, labels_new = process_videos_in_folder_parallel(r'dfdc_train_part_07\dfdc_train_part_7', r'dfdc_train_part_07\dfdc_train_part_7/metadata.json')

# Save the combined data to the .npy files
np.save('spatiotemporal_maps_new.npy', spatiotemporal_maps_new)
np.save('labels_new.npy', labels_new)



In [64]:
X_train = np.expand_dims(spatiotemporal_maps_new, axis=1)  # Adding time steps dimension
y_train = labels_new


In [69]:
print(len(spatiotemporal_maps_new))

0


In [70]:
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, LSTM, TimeDistributed, Input
from keras.optimizers import Adam
import numpy as np
from sklearn.model_selection import train_test_split

# Load the combined data
spatiotemporal_maps = np.load('spatiotemporal_maps.npy')
labels = np.load('labels.npy')

# Ensure spatiotemporal_maps has the shape (num_samples, height, width, channels)
if len(spatiotemporal_maps.shape) == 3:  # If it lacks the channels dimension
    spatiotemporal_maps = np.expand_dims(spatiotemporal_maps, axis=-1)

# Reshape spatiotemporal_maps to add the time_steps dimension
spatiotemporal_maps = np.expand_dims(spatiotemporal_maps, axis=1)

# Ensure labels are correctly shaped
labels = np.array(labels)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(spatiotemporal_maps, labels, test_size=0.2, random_state=42)

# Verify shapes
print("X_train shape:", X_train.shape)
print("X_val shape:", X_val.shape)
print("y_train shape:", y_train.shape)
print("y_val shape:", y_val.shape)

# Define the advanced model
def build_advanced_model(input_shape):
    input_layer = Input(shape=input_shape)
    
    # CNN for spatial features
    cnn = TimeDistributed(Conv2D(32, (3, 3), activation='relu'))(input_layer)
    cnn = TimeDistributed(MaxPooling2D((2, 2)))(cnn)
    cnn = TimeDistributed(Conv2D(64, (3, 3), activation='relu'))(cnn)
    cnn = TimeDistributed(MaxPooling2D((2, 2)))(cnn)
    cnn = TimeDistributed(Flatten())(cnn)
    
    # LSTM for temporal features
    lstm = LSTM(128, return_sequences=False)(cnn)
    
    # Fully connected layers
    dense = Dense(128, activation='relu')(lstm)
    dense = Dropout(0.5)(dense)
    output_layer = Dense(1, activation='sigmoid')(dense)
    
    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

# Set input shape based on the training data
input_shape = (X_train.shape[1], X_train.shape[2], X_train.shape[3], X_train.shape[4])
model = build_advanced_model(input_shape)

# Train the advanced model
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_val, y_val))


X_train shape: (1148, 1, 300, 50, 1)
X_val shape: (288, 1, 300, 50, 1)
y_train shape: (1148,)
y_val shape: (288,)
Epoch 1/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 21s 489ms/step - accuracy: 0.8217 - loss: 0.4519 - val_accuracy: 0.8333 - val_loss: 0.4502
Epoch 2/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 19s 519ms/step - accuracy: 0.8643 - loss: 0.4140 - val_accuracy: 0.8333 - val_loss: 0.4517
Epoch 3/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 19s 526ms/step - accuracy: 0.8690 - loss: 0.4030 - val_accuracy: 0.8333 - val_loss: 0.4520
Epoch 4/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 18s 504ms/step - accuracy: 0.8484 - loss: 0.4411 - val_accuracy: 0.8333 - val_loss: 0.4647
Epoch 5/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 19s 516ms/step - accuracy: 0.8655 - loss: 0.4173 - val_accuracy: 0.8333 - val_loss: 0.4516
Epoch 6/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 19s 532ms/step - accuracy: 0.8717 - loss: 0.3912 - val_accuracy: 0.8333 - val_loss: 0.4501
Epoch 7/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 19s 535ms/step - accuracy: 0.8495 - loss: 0.4347 - val_accuracy: 0.8333 - 

In [6]:
from tensorflow.keras.models import load_model
model = load_model(r'my_models\cnn_and_lstm.keras')

In [7]:
model.summary()

Model: "functional_92"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_17 (InputLayer)     │ (None, 1, 300, 50, 1)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_15             │ (None, 1, 298, 48, 32) │           320 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_16             │ (None, 1, 149, 24, 32) │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_17             │ (None, 1, 147, 22, 64) │        18,496 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_18             │ (None, 1, 73, 11, 64)  │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_19             │ (None, 1, 51392)       │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 128)            │    26,378,752 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_34 (Dense)                │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_35 (Dense)                │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 79,242,629 (302.29 MB)

 Trainable params: 26,414,209 (100.76 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 52,828,420 (201.52 MB)

In [10]:
def process_video(video_path, map_width=50, lowcut=0.8, highcut=2.5, fs=30, fixed_height=300, frame_skip=5, scale_factor=0.5):
    frames = extract_frames_and_detect_faces(video_path, frame_skip, scale_factor)
    if len(frames) == 0:
        return None
    green_channel_signal = extract_green_channel_signal(frames)
    filtered_signal = bandpass_filter(green_channel_signal, lowcut, highcut, fs)
    map_height = len(filtered_signal)
    spatiotemporal_map = create_spatiotemporal_map(filtered_signal, map_width, map_height)
    spatiotemporal_map = cv2.resize(spatiotemporal_map, (map_width, fixed_height))
    return spatiotemporal_map

def predict_video(video_path):
    spatiotemporal_map = process_video(video_path, fixed_height=300)
    if spatiotemporal_map is None:
        return 'Unable to process video'
    
    # Ensure correct shape and dimensions
    spatiotemporal_map = np.expand_dims(spatiotemporal_map, axis=0)  # Add batch dimension
    spatiotemporal_map = np.expand_dims(spatiotemporal_map, axis=1)  # Add channel dimension
    spatiotemporal_map = np.expand_dims(spatiotemporal_map, axis=-1)  # Add depth dimension
    
    # Predict using the model
    prediction = model.predict(spatiotemporal_map)
    confidence_score = prediction[0][0]
    label = 'Real' if prediction[0][0] > 0.5 else 'Fake'
    
    return label, confidence_score


# Example prediction
test_video_path = r'D:\kuruvadi.intern\OneDrive - Vivriti Capital Private Limited\Documents\DFD\train_sample_videos\eggbjzxnmg.mp4'
print(predict_video(test_video_path))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
('Fake', 0.13794237)


In [72]:
model.save('my_models/cnn_and_lstm.keras')